In [ ]:
!pip install transformers
!pip install emoji
!pip install langdetect

In [ ]:
import os

import numpy as np
print('numpy', np.__version__)
import pandas as pd
print('pandas', pd.__version__)
from langdetect import detect
import random
import re

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
#from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

from transformers import AutoTokenizer, BertTokenizer, BertForSequenceClassification

from tqdm import tqdm #progress bar

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

numpy 1.21.6
pandas 1.3.5


# https://www.analyticsvidhya.com/blog/2021/07/performing-sentiment-analysis-with-naive-bayes-classifier/

In [ ]:
from sklearn.model_selection import train_test_split
import joblib
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cd /content/drive/My Drive/Twitter

/content/drive/.shortcut-targets-by-id/16W2OVxf4pSh-3dRxed_aDRKcUuYcKpsj/Twitter


In [ ]:
data = pd.read_csv('all_tweets_english.tsv', delimiter='\t', index_col=0, on_bad_lines='skip')

In [ ]:
data['label'] = 0
data.loc[data['pos'] == 1,'label'] = 1
data.loc[data['neg'] == 1,'label'] = -1

In [ ]:
def preprocess_data(data):
    # Convert text to lowercase
    data['text'] = data['text'].str.replace('/srs',' ')
    data['text'] = data['text'].str.replace('/nbh',' ')
    data['text'] = data['text'].str.replace('/pos',' ')
    data['text'] = data['text'].str.replace('/hyp',' ')
    data['text'] = data['text'].str.replace('/neg',' ')
    return data

data = data.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

preprocess_data(data)
data.head()

,text,srs,nbh,hyp,pos,neg,label
tid,,,,,,,
1491924924386230287,RT @malewifealtair: @sobokify .,1,0,0,0,0,0
1491924858493620227,i just swallowed a small gemstone,1,0,0,1,0,1
1491924846242152457,@bonsorlol no bc it genuinely had great themes...,0,0,0,1,0,1
1491924808476422144,registering for classes i am so indecisive,0,0,0,0,1,-1
1491924691925311489,I found out about fnf a week or two after it c...,1,0,0,0,0,0


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
x = data['text']
y = data['label']
x, x_test, y, y_test = train_test_split(x,y, stratify=y, test_size=0.25, random_state=42)

In [ ]:
vec = CountVectorizer(stop_words='english', max_features = 12000)
x = vec.fit_transform(x).toarray()
x_test = vec.transform(x_test).toarray()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x, y)

MultinomialNB()

In [ ]:
model.score(x_test, y_test)

0.7022315946826118

In [ ]:
y_predicted = model.predict(x_test)

In [ ]:
import sklearn

sklearn.metrics.f1_score(y_test, y_predicted, average='micro')

0.7022315946826118

In [ ]:
sklearn.metrics.f1_score(y_test, y_predicted, average='weighted')

0.6895550725683741

In [ ]:
sklearn.metrics.f1_score(y_test, y_predicted, average='macro')

0.5807716646210817

In [ ]:
target_names = ['class 0', 'class 1', 'class 2']
print(sklearn.metrics.classification_report(y_test, y_predicted, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.41      0.22      0.29      3895
     class 1       0.69      0.65      0.67     13899
     class 2       0.74      0.83      0.78     19668

    accuracy                           0.70     37462
   macro avg       0.61      0.57      0.58     37462
weighted avg       0.69      0.70      0.69     37462

